Instalando Pyspark

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=466c065aad8a7fda560e5eb58eb9386d2ec55a88ca0d8991e2f113cef9cdcb4e
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


instanciando pyspark

In [5]:
#imports
from pyspark.sql.functions import mean,max,min,col,count
from pyspark.sql import SparkSession


In [2]:
#instanciando spark
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('spark IGTI') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

Leitura dos dados do enem 2019 e 2020 CSV

In [ ]:
# #lendo arquivo csv
# enem_2019 = (
#     spark
#     .read
#     .format('csv')
#     .option('header',True)
#     .option('inferSchema',True)
#     .option('delimiter',';')
#     .load('/content/drive/MyDrive/Documentos/cursos/igti/raw_data/enem/2019/*')
# )

enem_2020 = (
    spark
    .read
    .format('csv')
    .option('header',True)
    .option('inferSchema',True)
    .option('delimiter',';')
    .load('/content/drive/MyDrive/Documentos/cursos/igti/raw_data/enem/2020/*')
)

Escrita dos dados na camada stage em Parquet

In [12]:
#transformando os dados em formato parquet na camada stage
(
    enem_2019
    .write
    .mode('overwrite')
    .format('parquet')
    .partitionBy('NU_ANO')
    .save('/content/drive/MyDrive/Documentos/cursos/igti/stage/enem/2019')
)

(
    enem_2020
    .write
    .mode('overwrite')
    .format('parquet')
    .partitionBy('NU_ANO')
    .save('/content/drive/MyDrive/Documentos/cursos/igti/stage/enem/2020')
)

Leitura dos dados formato parquet

In [4]:
#lendo dados em formato parquet particionados
# enem_2019 = (
#     spark
#     .read
#     .format('parquet')
#     .option('header',True)
#     .option('inferSchema',True)
#     .option('delimiter',';')
#     .load('/content/drive/MyDrive/Documentos/cursos/igti/stage/enem/2019')
# )

enem_2020 = (
    spark
    .read
    .format('parquet')
    .option('header',True)
    .option('inferSchema',True)
    .option('delimiter',';')
    .load('/content/drive/MyDrive/Documentos/cursos/igti/stage/enem/2020')
)

amostragem dos dados

In [61]:
enem_2020.show(25)

+------------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+--------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------+
|NU_INSCRICAO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_

pergunta 5: 
Sobre o ENEM 2020, qual é a média da nota em matemática apenas para as alunas do sexo Feminino?

In [8]:
(
    enem_2020
    .groupby('TP_SEXO')
    .agg(
        mean(col('NU_NOTA_MT')).alias('MED_nota_MT'),
        count(col('TP_SEXO')).alias('CONTAGEM'),
        max(col('NU_NOTA_MT')).alias('MAX_NOTA_MT'),
        min(col('NU_NOTA_MT')).alias('MIN_NOTA_MT')
    )
  .show()
)

+-------+-----------------+--------+-----------+-----------+
|TP_SEXO|      MED_nota_MT|CONTAGEM|MAX_NOTA_MT|MIN_NOTA_MT|
+-------+-----------------+--------+-----------+-----------+
|      F|500.6864256785847| 3468805|      975.0|        0.0|
|      M|550.9087170619425| 2314304|      975.0|        0.0|
+-------+-----------------+--------+-----------+-----------+



pergunta6: Qual é a média da nota em Ciências Humanas para os alunos do sexo masculino que estudaram numa escola no estado de São Paulo?

In [17]:
#selecionando somente alunos do estado de sp
aluno_sp = enem_2020.filter('SG_UF_ESC == "SP"')
avg_science = (
     aluno_sp
    .groupby('TP_SEXO')
    .agg(
        mean(col('NU_NOTA_CH')).alias('avg_science')
        )
  .show()
)

+-------+-----------------+
|TP_SEXO|      avg_science|
+-------+-----------------+
|      F|536.6446700687505|
|      M|553.6780138704443|
+-------+-----------------+



Pergunta 7:  Qual é a média da nota em Ciências Humanas dos alunos que estudaram numa escola na cidade de Natal?

In [27]:
alunos_Natal = enem_2020.filter('NO_MUNICIPIO_ESC == "Natal"')
(
    alunos_Natal
    .agg(
        mean(col('NU_NOTA_CH')).alias('science_avg_natal')
    )
    .show()
)

+-----------------+
|science_avg_natal|
+-----------------+
|548.1773212818406|
+-----------------+



Pergunta8: Qual é o município (município da escola) que obteve a maior nota MÉDIA em matemática?

In [53]:
(
    enem_2020
    .groupBy('NO_MUNICIPIO_ESC')
    .agg(
        mean(col('NU_NOTA_MT')).alias('avg_math')
    )
).createTempView('tb_avg_math')
spark.sql('select * from tb_avg_math where tb_avg_math.avg_math > 700' ).show()

+-------------------+--------+
|   NO_MUNICIPIO_ESC|avg_math|
+-------------------+--------+
|Santana do Manhua�u|   720.2|
|     Pedro Teixeira|   703.4|
|          Biquinhas|   784.2|
|Santa Clara d'Oeste|   738.0|
|             Flora�|   702.7|
|       Dolcin�polis|   711.4|
|   Coqueiros do Sul|   787.4|
|     Novo Itacolomi|   823.2|
|             Vanini|   713.4|
|           Jesu�nia|   714.7|
|         Novo Xingu|   836.1|
+-------------------+--------+



 Pergunta 9: Quantas pessoas que estudaram numa escola em Recife fizeram a prova do ENEM nessa mesma cidade? (tirar duvida!!)

In [86]:
alunos_recife = enem_2020.filter('NO_MUNICIPIO_ESC == "Recife"')
alunos_recife.count()

10735

Pergunta 10: Qual é a nota média em Ciências Humanas dos alunos que estudaram numa escola no estado de Santa Catarina e possuem PELO MENOS 1 banheiro em casa?

In [87]:
alunos_SC = enem_2020.filter('SG_UF_ESC == "SC" and Q008 != "A"')
(
    alunos_SC
    .agg(
        mean(col('NU_NOTA_CH')).alias('avg_science_sc')
    )
    .show()
)


+-----------------+
|   avg_science_sc|
+-----------------+
|545.0439967817795|
+-----------------+



Pergunta 11: Qual é a nota média em matemática dos alunos cuja mãe possui PELO MENOS o ensino superior completo, do sexo feminino que estudaram numa escola em Belo Horizonte?

In [113]:
# and 
alunas_BH = enem_2020.filter('NO_MUNICIPIO_ESC == "Belo Horizonte" and TP_SEXO == "F"')
alunas_Mae_superior = alunas_BH.filter('Q002 == "F" or Q002 == "G"')
(
    alunas_Mae_superior
    .agg(
        mean(col('NU_NOTA_MT')).alias('avg_math')
    )
    .show()
)

+-----------------+
|         avg_math|
+-----------------+
|667.1279782501234|
+-----------------+

